In [1]:
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") # ignore python warnings of deprecation

df = pd.read_csv('../lending-club-loan-data/loan.csv', na_values=['#NAME?']) # '#NAME?' in the datafile will be converted to NaN

In [2]:
df.shape

(887379, 74)

In [3]:
df.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 74 columns):
id                             887379 non-null int64
member_id                      887379 non-null int64
loan_amnt                      887379 non-null float64
funded_amnt                    887379 non-null float64
funded_amnt_inv                887379 non-null float64
term                           887379 non-null object
int_rate                       887379 non-null float64
installment                    887379 non-null float64
grade                          887379 non-null object
sub_grade                      887379 non-null object
emp_title                      835917 non-null object
emp_length                     842554 non-null object
home_ownership                 887379 non-null object
annual_inc                     887375 non-null float64
verification_status            887379 non-null object
issue_d                        887379 non-null object
loan_status          

In [5]:
#Removing features which has got 75% null values
nulls = pd.DataFrame(round(df.isnull().sum()/len(df.index)*100,2),columns=['null_percent'])
drop_cols = nulls[nulls['null_percent']>75.0].index
df.drop(drop_cols, axis=1, inplace=True)

In [6]:
df.shape

(887379, 54)

In [7]:
df.head

<bound method NDFrame.head of               id  member_id  loan_amnt  funded_amnt  funded_amnt_inv  \
0        1077501    1296599     5000.0       5000.0      4975.000000   
1        1077430    1314167     2500.0       2500.0      2500.000000   
2        1077175    1313524     2400.0       2400.0      2400.000000   
3        1076863    1277178    10000.0      10000.0     10000.000000   
4        1075358    1311748     3000.0       3000.0      3000.000000   
5        1075269    1311441     5000.0       5000.0      5000.000000   
6        1069639    1304742     7000.0       7000.0      7000.000000   
7        1072053    1288686     3000.0       3000.0      3000.000000   
8        1071795    1306957     5600.0       5600.0      5600.000000   
9        1071570    1306721     5375.0       5375.0      5350.000000   
10       1070078    1305201     6500.0       6500.0      6500.000000   
11       1069908    1305008    12000.0      12000.0     12000.000000   
12       1064687    1298717     90

In [9]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'initial_list_status',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim'],
      dtype='object')

In [10]:
# Check missing values count and percent
total= df.isnull().sum().sort_values(ascending=False)
percent= (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)*100
missing_data= pd.concat([total, percent],axis=1, keys=["Total", "Percent"])
missing_data.head(54)

,Total,Percent
mths_since_last_delinq,454312,51.197065
next_pymnt_d,252971,28.507661
tot_cur_bal,70276,7.919502
total_rev_hi_lim,70276,7.919502
tot_coll_amt,70276,7.919502
emp_title,51462,5.799326
emp_length,44825,5.051393
last_pymnt_d,17659,1.990018
revol_util,502,0.056571
title,152,0.017129


In [11]:
# Any variable having missing values more than 50 percent are removed
df.drop([
         "emp_length", "next_pymnt_d", "last_pymnt_d", "mths_since_last_delinq",
         "revol_util", "collections_12_mths_ex_med","last_credit_pull_d",
         "earliest_cr_line","open_acc", "revol_util","pub_rec", "revol_bal", "funded_amnt",             
         "total_rev_hi_lim","emp_length", "emp_title", "tot_cur_bal" , "tot_coll_amt"], axis=1,
           inplace=True)



# Delete unwanted columns
df.drop(["url"], axis=1, inplace=True)

# Payment plan has all the values "n" and only 3 values "y" so it is not important
df.drop(["pymnt_plan"], axis=1, inplace=True)

# Since we have both address state and zip code let's drop zip code and use address state only
df.drop(["zip_code"], axis=1, inplace=True)

# Title is not important instead we will use "purpose" variable
df.drop(["title"], axis=1, inplace=True)

# The grade is implied by the subgrade, so let's drop the grade column.
#df.drop(["grade"], axis=1, inplace=True)

In [12]:
df.shape

(887379, 34)

In [13]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'purpose',
       'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'policy_code', 'application_type', 'acc_now_delinq'],
      dtype='object')

In [14]:
df.drop(["installment",        
"issue_d",       
"out_prncp" ,       
"out_prncp_inv",        
"total_rec_prncp",       
"total_rec_int",        
"total_rec_prncp",       
"total_rec_int",        
"total_rec_late_fee",       
"recoveries",        
"collection_recovery_fee",        
"last_pymnt_amnt",        
"policy_code",        
"application_type",       
"acc_now_delinq",        
"out_prncp",        
"out_prncp_inv",       
"initial_list_status"], axis=1, inplace=True)

In [15]:
df.shape

(887379, 20)

In [16]:
#Let's see the data shape and NaN values
print(df.shape)
print(type(df))
print(df.head())
print(df.isnull().sum())
print(df.isnull().sum().value_counts())

(887379, 20)
<class 'pandas.core.frame.DataFrame'>
        id  member_id  loan_amnt  funded_amnt_inv        term  int_rate grade  \
0  1077501    1296599     5000.0           4975.0   36 months     10.65     B   
1  1077430    1314167     2500.0           2500.0   60 months     15.27     C   
2  1077175    1313524     2400.0           2400.0   36 months     15.96     C   
3  1076863    1277178    10000.0          10000.0   36 months     13.49     C   
4  1075358    1311748     3000.0           3000.0   60 months     12.69     B   

  sub_grade home_ownership  annual_inc verification_status  loan_status  \
0        B2           RENT     24000.0            Verified   Fully Paid   
1        C4           RENT     30000.0     Source Verified  Charged Off   
2        C5           RENT     12252.0        Not Verified   Fully Paid   
3        C1           RENT     49200.0     Source Verified   Fully Paid   
4        B5           RENT     80000.0     Source Verified      Current   

          p

In [69]:
df.to_csv(r'C:\Users\Admin\Desktop\Data Science Methods and Tools\lending-club-loan-data\loan_preproccesed.csv', index = None, header=True)

In [11]:
#Filling the numeric columns with missing values by mean
for cols in df.columns:
    if df[cols].isnull().sum() != 0:
        df[cols].fillna((df[cols].mean()), inplace=True)
        
df.isnull().sum()

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [71]:
df.to_csv(r'C:\Users\Admin\Desktop\Data Science Methods and Tools\lending-club-loan-data\loan_prep.csv', index = None, header=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 20 columns):
id                     887379 non-null int64
member_id              887379 non-null int64
loan_amnt              887379 non-null float64
funded_amnt_inv        887379 non-null float64
term                   887379 non-null object
int_rate               887379 non-null float64
grade                  887379 non-null object
sub_grade              887379 non-null object
home_ownership         887379 non-null object
annual_inc             887379 non-null float64
verification_status    887379 non-null object
loan_status            887379 non-null object
purpose                887379 non-null object
addr_state             887379 non-null object
dti                    887379 non-null float64
delinq_2yrs            887379 non-null float64
inq_last_6mths         887379 non-null float64
total_acc              887379 non-null float64
total_pymnt            887379 non-null float64
total_py

In [19]:
#Checking for object columns
print(df.select_dtypes(['object']).columns)

Index(['term', 'grade', 'sub_grade', 'home_ownership', 'verification_status',
       'loan_status', 'purpose', 'addr_state'],
      dtype='object')


In [20]:
obj_cats = ['term', 'grade', 'sub_grade', 'home_ownership', 'verification_status',
       'loan_status', 'purpose', 'addr_state']

In [21]:
#Converting the object datatype of columns to category except date as it is datetime object
for colname in obj_cats:
    df[colname] = df[colname].astype('category')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 20 columns):
id                     887379 non-null int64
member_id              887379 non-null int64
loan_amnt              887379 non-null float64
funded_amnt_inv        887379 non-null float64
term                   887379 non-null category
int_rate               887379 non-null float64
grade                  887379 non-null category
sub_grade              887379 non-null category
home_ownership         887379 non-null category
annual_inc             887379 non-null float64
verification_status    887379 non-null category
loan_status            887379 non-null category
purpose                887379 non-null category
addr_state             887379 non-null category
dti                    887379 non-null float64
delinq_2yrs            887379 non-null float64
inq_last_6mths         887379 non-null float64
total_acc              887379 non-null float64
total_pymnt            887379 non-null 

In [77]:
from sklearn.preprocessing import OneHotEncoder

In [22]:
df.head()

,id,member_id,loan_amnt,funded_amnt_inv,term,int_rate,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,total_acc,total_pymnt,total_pymnt_inv
0,1077501,1296599,5000.0,4975.0,36 months,10.65,B,B2,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,27.65,0.0,1.0,9.0,5861.071414,5831.78
1,1077430,1314167,2500.0,2500.0,60 months,15.27,C,C4,RENT,30000.0,Source Verified,Charged Off,car,GA,1.00,0.0,5.0,4.0,1008.710000,1008.71
2,1077175,1313524,2400.0,2400.0,36 months,15.96,C,C5,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,8.72,0.0,2.0,10.0,3003.653644,3003.65
3,1076863,1277178,10000.0,10000.0,36 months,13.49,C,C1,RENT,49200.0,Source Verified,Fully Paid,other,CA,20.00,0.0,1.0,37.0,12226.302212,12226.30
4,1075358,1311748,3000.0,3000.0,60 months,12.69,B,B5,RENT,80000.0,Source Verified,Current,other,OR,17.94,0.0,0.0,38.0,3242.170000,3242.17


In [23]:
# Create a set of dummy variables from the term variable
df_term = pd.get_dummies(df['term'])

In [24]:
# Join the dummy variables to the main dataframe
df_new = pd.concat([df, df_term], axis=1)
df_new

,id,member_id,loan_amnt,funded_amnt_inv,term,int_rate,grade,sub_grade,home_ownership,annual_inc,...,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,total_acc,total_pymnt,total_pymnt_inv,36 months,60 months
0,1077501,1296599,5000.0,4975.000000,36 months,10.65,B,B2,RENT,24000.00,...,credit_card,AZ,27.65,0.0,1.0,9.0,5861.071414,5831.78,1,0
1,1077430,1314167,2500.0,2500.000000,60 months,15.27,C,C4,RENT,30000.00,...,car,GA,1.00,0.0,5.0,4.0,1008.710000,1008.71,0,1
2,1077175,1313524,2400.0,2400.000000,36 months,15.96,C,C5,RENT,12252.00,...,small_business,IL,8.72,0.0,2.0,10.0,3003.653644,3003.65,1,0
3,1076863,1277178,10000.0,10000.000000,36 months,13.49,C,C1,RENT,49200.00,...,other,CA,20.00,0.0,1.0,37.0,12226.302212,12226.30,1,0
4,1075358,1311748,3000.0,3000.000000,60 months,12.69,B,B5,RENT,80000.00,...,other,OR,17.94,0.0,0.0,38.0,3242.170000,3242.17,0,1
5,1075269,1311441,5000.0,5000.000000,36 months,7.90,A,A4,RENT,36000.00,...,wedding,AZ,11.20,0.0,3.0,12.0,5631.377753,5631.38,1,0
6,1069639,1304742,7000.0,7000.000000,60 months,15.96,C,C5,RENT,47004.00,...,debt_consolidation,NC,23.51,0.0,1.0,11.0,8136.840000,8136.84,0,1
7,1072053,1288686,3000.0,3000.000000,36 months,18.64,E,E1,RENT,48000.00,...,car,CA,5.35,0.0,2.0,4.0,3938.144334,3938.14,1,0
8,1071795,1306957,5600.0,5600.000000,60 months,21.28,F,F2,OWN,40000.00,...,small_business,CA,5.55,0.0,2.0,13.0,646.020000,646.02,0,1
9,1071570,1306721,5375.0,5350.000000,60 months,12.69,B,B5,RENT,15000.00,...,other,TX,18.08,0.0,0.0,3.0,1476.190000,1469.34,0,1


In [25]:
df_grade = pd.get_dummies(df['grade'])
df_subgrade = pd.get_dummies(df['sub_grade'])
df_homeowner = pd.get_dummies(df['home_ownership'])
df_verify = pd.get_dummies(df['verification_status'])
df_purp = pd.get_dummies(df['purpose'])
df_loan = pd.get_dummies(df['loan_status'])
df_address = pd.get_dummies(df['addr_state'])
df_newone = pd.concat([df, df_grade, df_subgrade, df_homeowner, df_verify, df_purp,df_loan, df_address ], axis =1)
df_newone

,id,member_id,loan_amnt,funded_amnt_inv,term,int_rate,grade,sub_grade,home_ownership,annual_inc,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,1077501,1296599,5000.0,4975.000000,36 months,10.65,B,B2,RENT,24000.00,...,0,0,0,0,0,0,0,0,0,0
1,1077430,1314167,2500.0,2500.000000,60 months,15.27,C,C4,RENT,30000.00,...,0,0,0,0,0,0,0,0,0,0
2,1077175,1313524,2400.0,2400.000000,36 months,15.96,C,C5,RENT,12252.00,...,0,0,0,0,0,0,0,0,0,0
3,1076863,1277178,10000.0,10000.000000,36 months,13.49,C,C1,RENT,49200.00,...,0,0,0,0,0,0,0,0,0,0
4,1075358,1311748,3000.0,3000.000000,60 months,12.69,B,B5,RENT,80000.00,...,0,0,0,0,0,0,0,0,0,0
5,1075269,1311441,5000.0,5000.000000,36 months,7.90,A,A4,RENT,36000.00,...,0,0,0,0,0,0,0,0,0,0
6,1069639,1304742,7000.0,7000.000000,60 months,15.96,C,C5,RENT,47004.00,...,0,0,0,0,0,0,0,0,0,0
7,1072053,1288686,3000.0,3000.000000,36 months,18.64,E,E1,RENT,48000.00,...,0,0,0,0,0,0,0,0,0,0
8,1071795,1306957,5600.0,5600.000000,60 months,21.28,F,F2,OWN,40000.00,...,0,0,0,0,0,0,0,0,0,0
9,1071570,1306721,5375.0,5350.000000,60 months,12.69,B,B5,RENT,15000.00,...,0,0,1,0,0,0,0,0,0,0


In [26]:
df_newone.shape

(887379, 146)

In [28]:
df_newone.drop(['term', 'grade','sub_grade' , 'home_ownership', 'verification_status', 'loan_status', 'purpose', 'addr_state'], inplace=True, axis=1)

In [29]:
df_newone.shape

(887379, 138)

In [30]:
df_newone.to_csv(r'C:\Users\Admin\Desktop\Data Science Methods and Tools\lending-club-loan-data\loan_cleaned.csv', index = None, header=True)

In [30]:
df_newone.head(5)
intRate=df_newone['int_rate']
print(df_newone.shape,'Bfore deleting column')
del df_newone['int_rate']
print(df_newone.shape,'after deng column')
df_newone['int_rate']=intRate
print(df_newone.shape,'after adding column')
X = df_newone.iloc[:, :-1].values
print(X.shape,'after slicing column')
y = df_newone.iloc[:, -1:].values
print(y.shape,'after last  column')
print(y)

(887379, 138) Bfore deleting column
(887379, 137) after deng column
(887379, 138) after adding column
(887379, 137) after slicing column
(887379, 1) after last  column
[[10.65]
 [15.27]
 [15.96]
 ...
 [15.99]
 [19.99]
 [11.99]]


In [100]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in c:\programdata\anaconda3\lib\site-packages (0.20.3)


You are using pip version 19.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [31]:
#Splitting the dataset into Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [32]:
df_newone.head(1)

,id,member_id,loan_amnt,funded_amnt_inv,annual_inc,dti,delinq_2yrs,inq_last_6mths,total_acc,total_pymnt,...,TN,TX,UT,VA,VT,WA,WI,WV,WY,int_rate
0,1077501,1296599,5000.0,4975.0,24000.0,27.65,0.0,1.0,9.0,5861.071414,...,0,0,0,0,0,0,0,0,0,10.65


In [33]:
from sklearn.linear_model import LinearRegression
#Fitting Simple Linear Regression to the training set
regressor = LinearRegression()
regressor.fit(X_train, y_train)
#Predicting the Test set result
y_pred = regressor.predict(X_test)

In [38]:
print(y_pred[0:10])

[[ 9.27355165]
 [ 7.14627054]
 [ 9.82184687]
 [12.58900366]
 [14.62819596]
 [12.07885362]
 [17.12028428]
 [12.90456545]
 [ 7.44990649]
 [ 5.01528059]]


In [39]:
print(y_test[0:10])

[[ 9.17]
 [ 5.99]
 [10.16]
 [12.99]
 [14.64]
 [11.99]
 [16.49]
 [12.69]
 [ 7.89]
 [ 5.32]]


In [118]:
import matplotlib.pyplot as plt

In [36]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [37]:
mean_absolute_percentage_error(y_test, y_pred)

3.0825154744416157

In [46]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 100, random_state = 0)
rf.fit(X_train,y_train)
#Predicting the new result
y_predict = rf.predict(X_test)

In [47]:
print(y_predict[0:10])

[ 9.17        5.9963     10.1105     12.95       14.6268     11.99
 16.49       12.69        7.89000243  5.32      ]


In [48]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [49]:
mean_absolute_percentage_error(y_test, y_predict)

MemoryError: 

In [50]:
from sklearn.model_selection import cross_val_score

In [53]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator = regressor,X =  X_train, y = y_train, cv=5)

In [54]:
scores.mean()

0.9810237779631311

In [55]:
scores.std()

0.00019043030950336129

In [ ]:
accuracy = cross_val_score(estimator = rf ,X =  X_train, y = y_train, cv=5)